In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

from src import get_segmented_image

In [3]:
ls

main.ipynb                     test_image_segmentation.ipynb


In [6]:
sigma=1
neighbor=8
K=5.0
min_comp_size=3000
input_file='../img/banana_vehicule.JPG'
output_file='../img/banana_vehicule2.JPG'

get_segmented_image(sigma, neighbor, K, min_comp_size, input_file, output_file)